# First order differential equations

In the following, we solve the following 1st order inhomogenous differential equation using Python's scipy Library. Let $A\in \mathbb{C}^{n \times n}$ with $n\in \mathbb{N}$

$\begin{align}
\frac{\partial^{n} }{\partial t^{n} }x(t) = A \, x(t) + u(t)\end{align}$

## Homogenous part

The following solves the following homogenous part of the equation:

$\begin{align}
\frac{\partial |\Psi|^{2}}{\partial t} + \text{div} \, \textbf{J} = 0 \\

\end{align}$ 

Which was to be shown. 

## Inhomogenous part
This part solves the inhomogenous part of the equation.

which was to be shown.